## Energy intake tracker

In [1]:
from tkinter import *
from PIL import ImageTk,Image
import sqlite3
import pandas as pd
from datetime import datetime

pd.options.display.max_rows = 100

In [2]:
root = Tk()
root.title('Energy tracker input')
root.iconbitmap('~/Desktop/jms.ico')
root.geometry("500x600")

# Databases

# Create a database or connect to one
conn = sqlite3.connect('energy_tracker.db')

# Create cursor
c = conn.cursor()

# # Create table
# c.execute("""CREATE TABLE food_intake (
# 		date date,
# 		food text,
# 		amount decimal
# 		)""")

# Create Update function to update a record
def update():
	# Create a database or connect to one
	conn = sqlite3.connect('energy_tracker.db')
	# Create cursor
	c = conn.cursor()

	record_id = delete_box.get()

	c.execute("""UPDATE food_intake SET
		date = :date,
		food = :food,
		amount = :amount
		WHERE oid = :oid""",
		{
		'date': date_editor.get(),
		'food': food_editor.get(),
		'amount': amount_editor.get(),
		'oid': record_id
		})


	#Commit Changes
	conn.commit()

	# Close Connection 
	conn.close()

	editor.destroy()
	root.deiconify()

# Create Edit function to update a record
def edit():
	root.withdraw()
	global editor
	editor = Tk()
	editor.title('Update A Record')
	editor.iconbitmap('~/Desktop/jms.ico')
	editor.geometry("400x300")
	# Create a database or connect to one
	conn = sqlite3.connect('energy_tracker.db')
	# Create cursor
	c = conn.cursor()

	record_id = delete_box.get()
	# Query the database
	c.execute("SELECT * FROM food_intake WHERE oid = " + record_id)
	records = c.fetchall()
	
	#Create Global Variables for text box names
	global date_editor
	global food_editor
	global amount_editor

	# Create Text Boxes
	date_editor = Entry(editor, width=30)
	date_editor.grid(row=0, column=1, padx=20, pady=(10, 0))
	food_editor = Entry(editor, width=30)
	food_editor.grid(row=1, column=1)
	amount_editor = Entry(editor, width=30)
	amount_editor.grid(row=2, column=1)

	
	# Create Text Box Labels
	date_label = Label(editor, text="Date")
	date_label.grid(row=0, column=0, pady=(10, 0))
	food_label = Label(editor, text="Food")
	food_label.grid(row=1, column=0)
	amount_label = Label(editor, text="Amount\n(based on 100 g)")
	amount_label.grid(row=2, column=0)

	# Loop thru results
	for record in records:
		date_editor.insert(0, record[0])
		food_editor.insert(0, record[1])
		amount_editor.insert(0, record[2])
	
	# Create a Save Button To Save edited record
	edit_btn = Button(editor, text="Save Record", command=update)
	edit_btn.grid(row=6, column=0, columnspan=2, pady=10, padx=10, ipadx=145)


# Create Function to Delete A Record
def delete():
	# Create a database or connect to one
	conn = sqlite3.connect('energy_tracker.db')
	# Create cursor
	c = conn.cursor()

	# Delete a record
	c.execute("DELETE from food_intake WHERE oid = " + delete_box.get())

	delete_box.delete(0, END)

	#Commit Changes
	conn.commit()

	# Close Connection 
	conn.close()



# Create Submit Function For database
def submit():
	# Create a database or connect to one
	conn = sqlite3.connect('energy_tracker.db')
	# Create cursor
	c = conn.cursor()

	# Insert Into Table
	c.execute("INSERT INTO food_intake VALUES (:date, :food, :amount)",
			{
				'date': date.get(),
				'food': food.get(),
				'amount': amount.get()
			})


	#Commit Changes
	conn.commit()

	# Close Connection 
	conn.close()

	# Clear The Text Boxes
	date.delete(0, END)
	food.delete(0, END)
	amount.delete(0, END)

# Create Query Function
def query():
	# Create a database or connect to one
	conn = sqlite3.connect('energy_tracker.db')
	# Create cursor
	c = conn.cursor()

	# Query the database
	c.execute("SELECT *, oid FROM food_intake")
	records = c.fetchall()
	# print(records)

	# Loop Thru Results
	print_records = ''
	for record in records:
		print_records += str(record[0]) + " " + str(record[1]) + " " + "\t" +str(record[6]) + "\n"

	query_label = Label(root, text=print_records)
	query_label.grid(row=12, column=0, columnspan=2)

	#Commit Changes
	conn.commit()

	# Close Connection 
	conn.close()


# Create Text Boxes
v = StringVar(root, value=datetime.now().date())

date = Entry(root, width=30, textvariable=v)
date.grid(row=0, column=1, padx=20, pady=(10, 0))
food = Entry(root, width=30)
food.grid(row=1, column=1)
amount = Entry(root, width=30)
amount.grid(row=2, column=1)
delete_box = Entry(root, width=30)
delete_box.grid(row=9, column=1, pady=5)

# Create Text Box Labels
date_label = Label(root, text="Date")
date_label.grid(row=0, column=0, pady=(10, 0))
food_label = Label(root, text="Food")
food_label.grid(row=1, column=0)
amount_label = Label(root, text="Amount\n(based on 100 g)")
amount_label.grid(row=2, column=0)
delete_box_label = Label(root, text="Select ID")
delete_box_label.grid(row=9, column=0, pady=5)

# Create Submit Button
submit_btn = Button(root, text="Add Record To Database", command=submit)
submit_btn.grid(row=6, column=0, columnspan=2, pady=10, padx=10, ipadx=100)

# Create a Query Button
query_btn = Button(root, text="Show Records", command=query)
query_btn.grid(row=7, column=0, columnspan=2, pady=10, padx=10, ipadx=137)

#Create A Delete Button
delete_btn = Button(root, text="Delete Record", command=delete)
delete_btn.grid(row=10, column=0, columnspan=2, pady=10, padx=10, ipadx=136)

# Create an Update Button
edit_btn = Button(root, text="Edit Record", command=edit)
edit_btn.grid(row=11, column=0, columnspan=2, pady=10, padx=10, ipadx=143)

#Commit Changes
conn.commit()

# Close Connection 
conn.close()

root.mainloop()

In [3]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("energy_tracker.db")
df = pd.read_sql_query("SELECT * from food_info", con)

# Verify that result of SQL query is stored in the dataframe
print(df.head())

con.close()

  food_name  energy  fat sugar
0    Banana     105    0  14.4
1     Apple      47    0      


In [4]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("energy_tracker.db")
df = pd.read_sql_query("SELECT * from MyFoodDataSpreadsheet", con)

# Verify that result of SQL query is stored in the dataframe
print(df.head(1))

con.close()

       ID                                               name   Food Group  \
0  167512  Pillsbury Golden Layer Buttermilk Biscuits Art...  Baked Foods   

   Calories  Fat (g)  Protein (g)  Carbohydrate (g)  Sugars (g)  Fiber (g)  \
0     307.0    13.24         5.88             41.18        5.88        1.2   

   Cholesterol (mg)  ...  Serving Description 5 (g)  Serving Weight 6 (g)  \
0               0.0  ...                       None                   NaN   

   Serving Description 6 (g)  Serving Weight 7 (g)  Serving Description 7 (g)  \
0                       None                   NaN                       None   

   Serving Weight 8 (g)  Serving Description 8 (g)  Serving Weight 9 (g)  \
0                   NaN                       None                   NaN   

   Serving Description 9 (g)  200 Calorie Weight (g)  
0                       None                  65.147  

[1 rows x 117 columns]


In [5]:
df.head()

,ID,name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),...,Serving Description 5 (g),Serving Weight 6 (g),Serving Description 6 (g),Serving Weight 7 (g),Serving Description 7 (g),Serving Weight 8 (g),Serving Description 8 (g),Serving Weight 9 (g),Serving Description 9 (g),200 Calorie Weight (g)
0,167512,Pillsbury Golden Layer Buttermilk Biscuits Art...,Baked Foods,307.0,13.24,5.88,41.18,5.88,1.2,0.0,...,None,NaN,None,NaN,None,NaN,None,NaN,None,65.147
1,167513,Pillsbury Cinnamon Rolls With Icing Refrigerat...,Baked Foods,330.0,11.27,4.34,53.42,21.34,1.4,0.0,...,None,NaN,None,NaN,None,NaN,None,NaN,None,60.606
2,167514,Kraft Foods Shake N Bake Original Recipe Coati...,Baked Foods,377.0,3.70,6.10,79.80,NaN,NaN,NaN,...,None,NaN,None,NaN,None,NaN,None,NaN,None,53.050
3,167515,George Weston Bakeries Thomas English Muffins,Baked Foods,232.0,1.80,8.00,46.00,NaN,NaN,NaN,...,None,NaN,None,NaN,None,NaN,None,NaN,None,86.207
4,167516,Waffles Buttermilk Frozen Ready-To-Heat,Baked Foods,273.0,9.22,6.58,41.05,4.30,2.2,15.0,...,None,NaN,None,NaN,None,NaN,None,NaN,None,73.260


In [6]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("energy_tracker.db")
df = pd.read_sql_query("SELECT * from suggested_energy_intake", con)

# Verify that result of SQL query is stored in the dataframe
print(df.head(1))

con.close()

  Nutrient  Daily requirements Units
0  Calorie              1500.0  kcal


In [7]:
df.head()

,Nutrient,Daily requirements,Units
0,Calorie,1500.0,kcal
1,Total fat,70.0,g
2,Saturated fat,20.0,g
3,Carbohydrate,260.0,g
4,Total sugars,90.0,g


In [8]:
# sugg_df = pd.read_csv("Suggested_nutritional_intake.csv", sep=";")

# sugg_df.head(20)

# result = sugg_df['Daily requirements'].str.split(' ', expand=True)

# result = result.loc[:,[0,1]]
# result.rename(columns={0:'amount', 1:'units'}, inplace=True)

# suggested_energy_intake = pd.concat([sugg_df[["Nutrient"]], result], axis=1)

# conn = sqlite3.connect("energy_tracker.db")
# suggested_energy_intake.to_sql("suggested_energy_intake", conn, if_exists='append', index=False)

In [9]:
stop!

SyntaxError: invalid syntax (<ipython-input-9-8c726f94684c>, line 1)

### Autocomplete entry tkinter example

In [ ]:
"""
Inspired by http://code.activestate.com/recipes/578253-an-entry-with-autocompletion-for-the-tkinter-gui/
Changes:
    - Fixed AttributeError: 'AutocompleteEntry' object has no attribute 'listbox'
    - Fixed scrolling listbox
    - Case-insensitive search
    - Added focus to entry field
    - Custom listbox length, listbox width matches entry field width
    - Custom matches function
"""

from tkinter import *
import re

class AutocompleteEntry(Entry):
    def __init__(self, autocompleteList, *args, **kwargs):

        # Listbox length
        if 'listboxLength' in kwargs:
            self.listboxLength = kwargs['listboxLength']
            del kwargs['listboxLength']
        else:
            self.listboxLength = 8

        # Custom matches function
        if 'matchesFunction' in kwargs:
            self.matchesFunction = kwargs['matchesFunction']
            del kwargs['matchesFunction']
        else:
            def matches(fieldValue, acListEntry):
                pattern = re.compile('.*' + re.escape(fieldValue) + '.*', re.IGNORECASE)
                return re.match(pattern, acListEntry)
                
            self.matchesFunction = matches

        
        Entry.__init__(self, *args, **kwargs)
        self.focus()

        self.autocompleteList = autocompleteList
        
        self.var = self["textvariable"]
        if self.var == '':
            self.var = self["textvariable"] = StringVar()

        self.var.trace('w', self.changed)
        self.bind("<Right>", self.selection)
        self.bind("<Up>", self.moveUp)
        self.bind("<Down>", self.moveDown)
        
        self.listboxUp = False

    def changed(self, name, index, mode):
        if self.var.get() == '':
            if self.listboxUp:
                self.listbox.destroy()
                self.listboxUp = False
        else:
            words = self.comparison()
            if words:
                if not self.listboxUp:
                    self.listbox = Listbox(width=self["width"], height=self.listboxLength)
                    self.listbox.bind("<Button-1>", self.selection)
                    self.listbox.bind("<Right>", self.selection)
                    self.listbox.place(x=self.winfo_x(), y=self.winfo_y() + self.winfo_height())
                    self.listboxUp = True
                
                self.listbox.delete(0, END)
                for w in words:
                    self.listbox.insert(END,w)
            else:
                if self.listboxUp:
                    self.listbox.destroy()
                    self.listboxUp = False
        
    def selection(self, event):
        if self.listboxUp:
            self.var.set(self.listbox.get(ACTIVE))
            self.listbox.destroy()
            self.listboxUp = False
            self.icursor(END)

    def moveUp(self, event):
        if self.listboxUp:
            if self.listbox.curselection() == ():
                index = '0'
            else:
                index = self.listbox.curselection()[0]
                
            if index != '0':                
                self.listbox.selection_clear(first=index)
                index = str(int(index) - 1)
                
                self.listbox.see(index) # Scroll!
                self.listbox.selection_set(first=index)
                self.listbox.activate(index)

    def moveDown(self, event):
        if self.listboxUp:
            if self.listbox.curselection() == ():
                index = '0'
            else:
                index = self.listbox.curselection()[0]
                
            if index != END:                        
                self.listbox.selection_clear(first=index)
                index = str(int(index) + 1)
                
                self.listbox.see(index) # Scroll!
                self.listbox.selection_set(first=index)
                self.listbox.activate(index) 

    def comparison(self):
        return [ w for w in self.autocompleteList if self.matchesFunction(self.var.get(), w) ]

if __name__ == '__main__':
    autocompleteList = [ 'Dora Lyons (7714)', 'Hannah Golden (6010)', 'Walker Burns (9390)', 'Dieter Pearson (6347)', 'Allen Sullivan (9781)', 'Warren Sullivan (3094)', 'Genevieve Mayo (8427)', 'Igor Conner (4740)', 'Ulysses Shepherd (8116)', 'Imogene Bullock (6736)', 'Dominique Sanchez (949)', 'Sean Robinson (3784)', 'Diana Greer (2385)', 'Arsenio Conrad (2891)', 'Sophia Rowland (5713)', 'Garrett Lindsay (5760)', 'Lacy Henry (4350)', 'Tanek Conley (9054)', 'Octavia Michael (5040)', 'Kimberly Chan (1989)', 'Melodie Wooten (7753)', 'Winter Beard (3896)', 'Callum Schultz (7762)', 'Prescott Silva (3736)', 'Adena Crane (6684)', 'Ocean Schroeder (2354)', 'Aspen Blevins (8588)', 'Allegra Gould (7323)', 'Penelope Aguirre (7639)', 'Deanna Norman (1963)', 'Herman Mcintosh (1776)', 'August Hansen (547)', 'Oscar Sanford (2333)', 'Guy Vincent (1656)', 'Indigo Frye (3236)', 'Angelica Vargas (1697)', 'Bevis Blair (4354)', 'Trevor Wilkinson (7067)', 'Kameko Lloyd (2660)', 'Giselle Gaines (9103)', 'Phyllis Bowers (6661)', 'Patrick Rowe (2615)', 'Cheyenne Manning (1743)', 'Jolie Carney (6741)', 'Joel Faulkner (6224)', 'Anika Bennett (9298)', 'Clayton Cherry (3687)', 'Shellie Stevenson (6100)', 'Marah Odonnell (3115)', 'Quintessa Wallace (5241)', 'Jayme Ramsey (8337)', 'Kyle Collier (8284)', 'Jameson Doyle (9258)', 'Rigel Blake (2124)', 'Joan Smith (3633)', 'Autumn Osborne (5180)', 'Renee Randolph (3100)', 'Fallon England (6976)', 'Fallon Jefferson (6807)', 'Kevyn Koch (9429)', 'Paki Mckay (504)', 'Connor Pitts (1966)', 'Rebecca Coffey (4975)', 'Jordan Morrow (1772)', 'Teegan Snider (5808)', 'Tatyana Cunningham (7691)', 'Owen Holloway (6814)', 'Desiree Delaney (272)', 'Armand Snider (8511)', 'Wallace Molina (4302)', 'Amela Walker (1637)', 'Denton Tillman (201)', 'Bruno Acevedo (7684)', 'Slade Hebert (5945)', 'Elmo Watkins (9282)', 'Oleg Copeland (8013)', 'Vladimir Taylor (3846)', 'Sierra Coffey (7052)', 'Holmes Scott (8907)', 'Evelyn Charles (8528)', 'Steel Cooke (5173)', 'Roth Barrett (7977)', 'Justina Slater (3865)', 'Mara Andrews (3113)', 'Ulla Skinner (9342)', 'Reece Lawrence (6074)', 'Violet Clay (6516)', 'Ainsley Mcintyre (6610)', 'Chanda Pugh (9853)', 'Brody Rosales (2662)', 'Serena Rivas (7156)', 'Henry Lang (4439)', 'Clark Olson (636)', 'Tashya Cotton (5795)', 'Kim Matthews (2774)', 'Leilani Good (5360)', 'Deirdre Lindsey (5829)', 'Macy Fields (268)', 'Daniel Parrish (1166)', 'Talon Winters (8469)' ]
    def matches(fieldValue, acListEntry):
        pattern = re.compile(re.escape(fieldValue) + '.*', re.IGNORECASE)
        return re.match(pattern, acListEntry)
    
    root = Tk()
    entry = AutocompleteEntry(autocompleteList, root, listboxLength=6, width=32, matchesFunction=matches)
    entry.grid(row=0, column=0)    
    Button(text='Python').grid(column=0)
    Button(text='Tkinter').grid(column=0)
    Button(text='Regular Expressions').grid(column=0)
    Button(text='Fixed bugs').grid(column=0)
    Button(text='New features').grid(column=0)
    Button(text='Check code comments').grid(column=0)
    root.mainloop()